## 요리/레시피 데이터 수집
- 수집할 데이터 : 요리이름, 카테고리, 난이도, 소요시간, 재료, 요리방법, 요리설명, 링크, 이미지링크
- 수집 대상 사이트 : 우리의 식탁(https://wtable.co.kr/recipes)
- 수집 방법 : 웹 크롤링(스크래핑)
    - BeautifulSoup
    - requests

### 라이브러리 import

In [ ]:
# 데이터 분석
import numpy as np
import pandas as pd

# 웹 스크래핑
import requests
from bs4 import BeautifulSoup as bs

# HTTP 클라이언트
import urllib3
urllib3.disable_warnings()

# 진행시간 표시
import time
from tqdm.notebook import tqdm
tqdm.pandas()

### 웹 스크래핑 진행하기

In [ ]:
def get_link(token):
    "음식 상세정보 링크 반환"
    link = f'https://wtable.co.kr/recipes/{token}?location=recipe_home'
    return link

In [ ]:
# 카테고리 코드
category = {'메인요리': 1,
            '초대요리': 2,
            '간식': 5,
            '간단요리': 126,
            '밑반찬': 183,
            '채식': 184,
            '한식': 185,
            '양식': 186,
            '일식': 187,
            '중식': 188,
            '퓨전': 189,
            '분식': 190,
            '안주': 191,
            '베이킹': 192,
            '다이어트': 193,
            '도시락': 214,
            '키토': 242,
            '오븐 요리': 273
            }

In [ ]:
def get_one_page(cuisine, start):
    "cuisine의 start번째부터 40개 수집"
    url_base = f'https://wtable.net/api_v2/theme/recipe/list?&platform=web&'
    url_base2 = f'theme_id={category[cuisine]}&offset={start}&limit=40'
    url = url_base + url_base2

    response = requests.get(url, verify=False)
    data = response.json()['data']

    level_list = list(map(lambda x: x['level'], data))
    time_list = list(map(lambda x: x['time'], data))
    title_list = list(map(lambda x: x['title'], data))
    link_list = [get_link(i['token']) for i in data]
    img_list = list(map(lambda x: x['profile_img'], data))


    df = pd.DataFrame({'요리':title_list,                 
                       '종류':cuisine,
                       '난이도':level_list,
                       '소요시간':time_list,
                       '링크':link_list,
                       '사진':img_list})

    return df

In [ ]:
def get_all_pages(cuisine):
    "모든 페이지 수집"
    start = 0
    page_list = []
    while True:
        df_one_page = get_one_page(cuisine, start)
        if df_one_page.empty:
            break
        else:
            page_list.append(df_one_page)
            time.sleep(0.01)
            start += 40

    df_all_page = pd.concat(page_list, ignore_index=True)
    
    return df_all_page

In [ ]:
# 모든 카테고리를 대상으로 get_all_pages 적용하기
cuisines = []
for i in category:
    cuisines.append(get_all_pages(i))
result = pd.concat(cuisines, ignore_index=True)

In [ ]:
def get_details(url):
    "링크에서 재료와 요리방법, 설명 수집 함수"
    try: 
        response = requests.get(url, verify=False)
        soup = bs(response.text)    
        tmp = soup.find_all('div', {'class':'Text__Description02-sc-1qy6bx2-0 fCbbYE'})
        ingredients = [i.find_all('div')[0].text for i in tmp]
        tmp2 = soup.find_all('p', {'class':'Text__Pre01-sc-1qy6bx2-2 enJPxd'})
        cook = [i.text for i in tmp2]
        description = soup.find('p', {'class':'RecipeDetailstyle__Description-q7sykd-14 IdQIJ'}).text
        return ingredients, cook, description
    
    except:
        return None, None, None

In [ ]:
# 링크에서 재료와 요리방법, 설명 가져오기
details = result['링크'].progress_map(get_details)

  0%|          | 0/4340 [00:00<?, ?it/s]

In [ ]:
result['재료'] = [i[0] for i in details]
result['요리방법'] = [i[1] for i in details]
result['설명'] = [i[2] for i in details]

### 수집한 데이터 확인하기

In [ ]:
# 수집한 데이터 구성 요약
result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4340 entries, 0 to 4339
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   요리      4340 non-null   object
 1   종류      4340 non-null   object
 2   난이도     4340 non-null   int64 
 3   소요시간    4340 non-null   int64 
 4   링크      4340 non-null   object
 5   사진      4340 non-null   object
 6   재료      4340 non-null   object
 7   요리방법    4340 non-null   object
 8   설명      4340 non-null   object
dtypes: int64(2), object(7)
memory usage: 305.3+ KB


In [ ]:
# 요리, 종류, 설명만 가져와서 보기
result[['요리', '종류', '설명']]

,요리,종류,설명
0,꼬시래기 물냉면,메인요리,"꼬시래기는 해조류의 한 종류인데요. 지방, 탄수화물 함량이 낮고 칼슘과 식이섬유를 ..."
1,닭죽,메인요리,아플 때나 기운이 없을 때 간단히 끓여 먹기 좋은 닭죽! 맛이 좋은 건 물론 속을 ...
2,생채소 참치비빔밥,메인요리,별다른 반찬이 없을 때 해 먹기 딱 좋은 생채소 참치비빔밥 레시피 소개할게요. 집에...
3,등갈비강정,메인요리,돼지고기 등갈비는 야들야들한 살코기에 육즙이 가득 스며들어 뜯어 먹는 재미까지 있어...
4,앤초비 파스타,메인요리,앤초비는 지중해나 유럽 근해에서 나는 멸치류의 작은 물고기를 발효시킨 젓갈입니다. ...
...,...,...,...
4335,돼지고기 김치찜,오븐 요리,부드럽게 익은 묵은지에 삼겹살을 함께 먹으면 환상의 궁합이죠! 남녀노소 모두 좋아하...
4336,닭고기 덮밥,오븐 요리,쫄깃한 닭다리살과 은은하게 매운 꽈리고추가 정말 잘 어울리는 덮밥. 간편하고 근사한...
4337,감자 버터 오븐구이,오븐 요리,"맛있을 수밖에 없는 메뉴! 담백한 감자에 짭조름한 치즈, 베이컨까지 어우러진 감자 ..."
4338,고등어 강정,오븐 요리,"고등어를 색다르게 즐기는 요리. 비린 맛이 하나도 없고, 매콤 달콤한 소스와 너무 ..."


In [ ]:
# 데이터 미리보기
result.head(1)

,요리,종류,난이도,소요시간,링크,사진,재료,요리방법,설명
0,꼬시래기 물냉면,메인요리,1,20,https://wtable.co.kr/recipes/GdRMuEBF9nXVJAtqw...,https://static.wtable.co.kr/image/production/s...,"[꼬시래기, 시판용 냉면육수, 오이, 삶은 달걀, 쌈무, 통깨, 소금, 식초, 고추...","[끓는 물에 소금, 식초, 꼬시래기를 넣어 30초 정도 데쳐주세요.\r\n(tip....","꼬시래기는 해조류의 한 종류인데요. 지방, 탄수화물 함량이 낮고 칼슘과 식이섬유를 ..."


### 수집한 데이터 저장하기

In [ ]:
result.to_csv('data/raw_recipes.csv', index=False)

In [ ]:
raw_recipes = pd.read_csv('data/raw_recipes.csv')
raw_recipes.head()

,요리,종류,난이도,소요시간,링크,사진,재료,요리방법,설명
0,꼬시래기 물냉면,메인요리,1,20,https://wtable.co.kr/recipes/GdRMuEBF9nXVJAtqw...,https://static.wtable.co.kr/image/production/s...,"['꼬시래기', '시판용 냉면육수', '오이', '삶은 달걀', '쌈무', '통깨'...","['끓는 물에 소금, 식초, 꼬시래기를 넣어 30초 정도 데쳐주세요.\r\n(tip...","꼬시래기는 해조류의 한 종류인데요. 지방, 탄수화물 함량이 낮고 칼슘과 식이섬유를 ..."
1,닭죽,메인요리,1,30,https://wtable.co.kr/recipes/QawUyVgt3e5wvCT9d...,https://static.wtable.co.kr/image/production/s...,"['찹쌀밥', '닭가슴살', '다진 당근', '통마늘', '물', '소금', '참기...","['냄비에 물과 닭가슴살, 마늘, 대파를 넣고 15분간 삶아주세요. ', '삶은 닭...",아플 때나 기운이 없을 때 간단히 끓여 먹기 좋은 닭죽! 맛이 좋은 건 물론 속을 ...
2,생채소 참치비빔밥,메인요리,1,20,https://wtable.co.kr/recipes/VxEeT7YiZ5QQETGmj...,https://static.wtable.co.kr/image/production/s...,"['밥', '참치캔(85g)', '상추', '당근', '깻잎', '양파', '새싹 ...","['참치는 꽉 짜서 기름을 빼주세요. ', '채소는 가볍게 씻어 채반에서 물기를 빼...",별다른 반찬이 없을 때 해 먹기 딱 좋은 생채소 참치비빔밥 레시피 소개할게요. 집에...
3,등갈비강정,메인요리,2,40,https://wtable.co.kr/recipes/1nyadXLMLxWsNiL3y...,https://static.wtable.co.kr/image/production/s...,"['돼지 등갈비', '파채', '물', '청주', '통후추', '생강', '땅콩가루...",['등갈비는 찬물에 30분 정도 담가 핏물을 뺀 후 흐르는 물에 깨끗하게 씻어주세요...,돼지고기 등갈비는 야들야들한 살코기에 육즙이 가득 스며들어 뜯어 먹는 재미까지 있어...
4,앤초비 파스타,메인요리,1,20,https://wtable.co.kr/recipes/PJwQfoHitXrpY8n3B...,https://static.wtable.co.kr/image/production/s...,"['파스타 면', '앤초비', '케이퍼', '마늘', '루꼴라', '면수', '올리...","['앤초비와 케이퍼는 작게 다지고 마늘은 편으로 썰어주세요.', '냄비에 물과 소금...",앤초비는 지중해나 유럽 근해에서 나는 멸치류의 작은 물고기를 발효시킨 젓갈입니다. ...
